In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit import Aer
import numpy as np
from sklearn.model_selection import train_test_split
from qiskit.aqua.algorithms import SklearnSVM
from sklearn.model_selection import train_test_split
from qiskit import BasicAer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

In [22]:
class preprocessing:
    def __init__(self, df, feature_cols, prediction_column,n_components):
        self.df = df
        self.feature_cols = feature_cols
        self.prediction_column = prediction_column
        self.n_components = n_components
    def sample(self):
        df = self.df.sample(frac = 1)        
        return df
    def scaling_average(self):
        for column in self.feature_cols:
            df[column] = (self.df[column]-self.df[column].min())/(self.df[column].max()-self.df[column].min())
        return df
    def scaling_Guassian(self):
        for column in self.feature_cols:
            df[column] = (self.df[column] - self.df[column].mean())/(self.df[column].std())
        return df
    def normalize(self):
        for column in self.feature_cols:
            df[column] = (self.df[column])/(self.df[column].max())
        return df
        
    def xy_split(self):
        X = []
        y = []
        for i in range(0,len(self.df)):
            columns = []
            columns_y = []
            for column in self.feature_cols:
                columns.append(self.df[column][i])
            columns_y.append(self.df[self.prediction_column][i])
            X.append(columns)
            y.append(columns_y[0])
        return X, y
    def pca(self,X):
        pca = PCA(n_components=self.n_components).fit(X)
        X = pca.transform(X)
        return X
class test_train:
    def __init__(self, X, Y, total_size, test_ratio):
        self.x = X
        self.y = Y
        self.total_size = total_size
        self.test_size = test_ratio
    def test_train_split(self):
        X = np.array(self.x[0:self.total_size])
        y = np.array(self.y[0:self.total_size])
        train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=self.test_size)
        return train_x, test_x, train_y, test_y
    def test_train_datasets(self,train_x, test_x, train_y, test_y):
        training_dataset={'A':train_x[train_y==0],'B':train_x[train_y==1]}
        test_dataset={'A':test_x[test_y==0],'B':test_x[test_y==1]}
        return training_dataset,  test_dataset
    
class prediction:
    def __init__(self,x,y,total_size,num_prediction_points):
        self.total_ = df
        self.x = x
        self.y = y
        self.num = num_prediction_points
    def prediction_dataset(self):
        x = np.array(self.x[total_size+1:total_size+self.num+1])
        y = np.array(self.y[total_size+1:total_size+self.num+1])
        datapoints={'A':x[y==0],'B':x[y==1]}
        datapoints, class_to_label = split_dataset_to_data_and_labels(datapoints)
        return datapoints, class_to_label

class quantum_simulator:
    def __init__(self,n_features,reps,simulator,entanglement,training_dataset,test_dataset,datapoints,seed,shots):
        self.n_features = n_features
        self.reps = reps
        self.simulator = simulator
        self.entanglement = entanglement
        self.training_dataset = training_dataset
        self.test_dataset = test_dataset
        self.datapoints = datapoints
        self.seed = seed
        self.shots = shots
    def simulate(self):
        feature_map = ZZFeatureMap(feature_dimension=self.n_features, reps=self.reps, entanglement=self.entanglement)
        qsvm = QSVM(feature_map, self.training_dataset, self.test_dataset, self.datapoints)
        backend = BasicAer.get_backend(self.simulator)
        quantum_instance = QuantumInstance(backend, shots=self.shots, seed_simulator=self.seed, seed_transpiler=self.seed)
        result = qsvm.run(quantum_instance)
        return result

In [3]:
df = pd.read_csv('/Users/sreramanm/Desktop/docs/Bank_Personal_Loan_Modelling.csv')
feature_cols = ['Age','Income', 'Family', 'CCAvg', 'Education', 'Mortgage','Securities Account','CD Account','Online','CreditCard']
prediction_column = 'Personal Loan'
total_size = 100
test_ratio = 0.2
n_components = len(feature_cols)
reps = 2
simulator = 'qasm_simulator'
entanglement = 'linear'
seed = 10598
shots = 1024
#Shuffle the dataset
df = preprocessing(df, feature_cols, prediction_column,n_components).sample()
#Scaling function
df = preprocessing(df, feature_cols, prediction_column,n_components).scaling_average()
#normalization
df = preprocessing(df, feature_cols, prediction_column,n_components).normalize()
#x,y-split
X,y = preprocessing(df, feature_cols, prediction_column,n_components).xy_split()
#test-train functions
train_x, test_x, train_y, test_y=test_train(X,y,total_size,test_ratio).test_train_split()
training_dataset,  test_dataset = test_train(X,y,total_size,test_ratio).test_train_datasets(train_x, test_x, train_y, test_y)
#Define the datapoints for prediction
datapoints,class_to_label = prediction(X,y,total_size,10).prediction_dataset()
#Qasm simulator
result = quantum_simulator(n_components,reps,simulator,entanglement,training_dataset,test_dataset,datapoints[0],seed,shots).simulate()
result_svm = SklearnSVM(training_dataset, test_dataset, datapoints[0]).run()
#Print the results
predicted_labels_svm = result_svm["predicted_labels"]
predicted_labels = result["predicted_labels"]
df = pd.DataFrame(columns=['QSVM', 'SVM'])
data = {'QSVM': [result["testing_accuracy"],100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)],
        'SVM': [result_svm["testing_accuracy"], 100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)]}
df = pd.DataFrame(data, index=['Testing success ratio',
                               'Prediction success rate'])
df

,QSVM,SVM
Testing success ratio,1.0,0.95
Prediction success rate,100.0,100.00


In [26]:
df = pd.read_csv('/Users/sreramanm/Desktop/docs/Social_Network_Ads.csv').replace("Male",1).replace("Female",2)
feature_cols = ['Age','EstimatedSalary','Gender']
prediction_column = 'Purchased'
total_size = 120
test_ratio = 0.25
n_components = len(feature_cols)
reps = 2
simulator = 'qasm_simulator'
entanglement = 'linear'
seed = 10598
shots = 1024
df = preprocessing(df, feature_cols, prediction_column,n_components).sample()
df = preprocessing(df, feature_cols, prediction_column,n_components).scaling_average()
df = preprocessing(df, feature_cols, prediction_column,n_components).normalize()
X,y = preprocessing(df, feature_cols, prediction_column,n_components).xy_split()
X = preprocessing(df, feature_cols, prediction_column,n_components).pca(X)
train_x, test_x, train_y, test_y=test_train(X,y,total_size,test_ratio).test_train_split()
training_dataset,  test_dataset = test_train(X,y,total_size,test_ratio).test_train_datasets(train_x, test_x, train_y, test_y)
datapoints,class_to_label = prediction(X,y,total_size,20).prediction_dataset()
result = quantum_simulator(n_components,reps,simulator,entanglement,training_dataset,test_dataset,datapoints[0],seed,shots).simulate()
#Tune SVM parameters
total_size = 120
test_ratio = 0.24
svc_model = SVC(C= .1, kernel='linear', gamma= 1)
svc_model.fit(train_x, train_y)
prediction_test = svc_model.predict(test_x)
prediction_datapoints = svc_model.predict(datapoints[0])
df = pd.DataFrame(columns=['QSVM', 'SVM'])
data = {'QSVM': [result["testing_accuracy"],100*np.count_nonzero(predicted_labels == datapoints[1])/len(predicted_labels)],
        'SVM': [metrics.accuracy_score(test_y, prediction_test), 100*metrics.accuracy_score(datapoints[1], prediction_datapoints)]}
df = pd.DataFrame(data, index=['Testing success ratio',
                               'Prediction success rate'])
df

,QSVM,SVM
Testing success ratio,0.966667,0.833333
Prediction success rate,95.000000,95.000000
